# Strategy Benchmark

In [ ]:
from backtest import *
from data_preparation import *
import pandas as pd

## Strategy Demo

In [ ]:
period = "35d"
interval = "5m"
ticker_list = ['PEP']
ticker = ticker_list[0]

In [ ]:
init_balance = 10000

df, last_update = refresh(ticker, period=period, interval = interval)
bnh = buy_n_hold(init_balance, df)

In [ ]:
import strategy.strategy_wma as wma

df_wma = wma.prep_data(df, 89, 90)
test(init_balance, df_wma, wma.sell_or_buy, output="oneline", verbose=False)
plot_wma(df_wma, signals=buy_sell(df_wma, wma.sell_or_buy))

In [ ]:
#import strategy.strategy_hma as hma

#df_hma = hma.prep_data(df, 55, 34)
#a = test(init_balance, df_hma, hma.sell_or_buy)
#plot_hma(df_hma, signals=buy_sell(df_hma, hma.sell_or_buy))

In [ ]:
import strategy.strategy_macd as macd

df_macd = macd.prep_data(df, 144, 377)
test(init_balance, df_macd, macd.sell_or_buy, verbose = False)
plot_macd(df_macd, signals=buy_sell(df_macd, macd.sell_or_buy))

In [ ]:
#import strategy.strategy_kagi as kagi

#df_kagi = kagi.prep_data(df, 0.01, 1)
#test(init_balance, df_kagi, kagi.sell_or_buy, verbose=False)
#kagi.plot(df, True)

In [ ]:
import strategy.strategy_supertrend as supertrend

st_df = supertrend.prep_data(df, 3, 377)
test(init_balance, st_df, supertrend.sell_or_buy)
plot_supertrend(st_df, signals=buy_sell(st_df, supertrend.sell_or_buy))

In [ ]:
import strategy.strategy_rvwma as rvwma

df_rvwma = rvwma.prep_data(df, 55, 378)
test(init_balance, df_rvwma, rvwma.sell_or_buy)
plot_rvwma(df_rvwma, signals=buy_sell(df_rvwma, rvwma.sell_or_buy))

## Parameter Tuning

In [ ]:
ticker_list =["TSLA", "AAPL", "MSFT", "GOOGL", "BRK-B"]

fib_seq = [13, 21, 34, 55, 89, 144, 233, 377, 610]
fib1_seq = [14, 22, 35, 56, 90, 145, 234, 378, 611]
reversal = [0.001, 0.002, 0.005, 0.0075, 0.01, 0.02, 0.03, 0.04, 0.05]
multipiler = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6.0]

In [ ]:
picks = []
picks += run_strategy(ticker_list, fib_seq, fib1_seq, period, interval, wma, False)
picks += run_strategy(ticker_list, fib_seq, fib1_seq, period, interval, macd, False)
picks += run_strategy(ticker_list, multipiler, fib_seq, period, interval, supertrend)
picks += run_strategy(ticker_list, fib_seq, fib1_seq, period, interval, rvwma)

print("Done!")

In [ ]:
print(f"Total number of picks: {len(picks)}")

picks = [pick for pick in picks if pick[2] < 10]
picks.sort(reverse=True, key=lambda i: i[1])

for i in range(0, min(len(picks), 10)):
    print(
        "  The #{} config is ({}) profit margin is {:.4f}%. # of Trade is {}".format(
            i + 1,
            picks[i][0],
            picks[i][1],
            picks[i][2])
        )

## Verification

In [ ]:
#tickers = ['TQQQ','AMD', 'W', 'UAL']

for ticker in ticker_list:
    df, last_update = refresh(ticker, period, interval)
    
    print("Testing '{}'...".format(ticker))
    print("    WMA: \t",end='')
    test(init_balance, wma.prep_data(df, 233, 90), wma.sell_or_buy) # verified
    print("    MACD: \t",end='')
    test(init_balance, macd.prep_data(df, 34, 56), macd.sell_or_buy) # verified
    print("    SuperT: \t",end='')
    test(init_balance, supertrend.prep_data(df, 6.0, 55), supertrend.sell_or_buy) # verified
    print("    RVWMA: \t",end='')
    test(init_balance, rvwma.prep_data(df, 55, 378), rvwma.sell_or_buy) # verified
  
    ################################################################
    print("    G Best: \t",end='')
    test(init_balance, rd.prep_data(df, 0.03, 233), rd.sell_or_buy) # verified

    #print("    HMA: \t",end='')
    #test(10000, hma.prep_data(df, 89, 35), hma.sell_or_buy) # verified
    #print("    Kagi: \t",end='')
    #test(10000, kagi.prep_data(df, 0.005, 1), kagi.sell_or_buy) # verified
    print("    Buy&Hold: \t",end='')
    buy_n_hold(init_balance, df)